In [ ]:
import lib.areamanager as areamanager
import pandas as pd
import json
import time
import collections
import numpy as np
SPLIT_YEAR=2017

cities=['lasvegas','phoenix','charlotte','madison']

In [ ]:
fbusiness=open("../data/business.json")
poi_data = dict()
start_time=time.time()
for i, line in enumerate(fbusiness):  
    # json to dict
    obj_json = json.loads(line)
    # add to the data collection
    if obj_json['categories'] != None:
        poi_data[obj_json['business_id']]={'latitude':obj_json['latitude'],
                         'longitude':obj_json['longitude'],
                         'categories':obj_json['categories'].split(', ')}
    else:
        poi_data[obj_json['business_id']]={'latitude':obj_json['latitude'],
                 'longitude':obj_json['longitude'],
                 'categories':obj_json['categories']}
poi_data_list=list(poi_data)
print(time.time()-start_time)


In [ ]:
areas=dict()
for city in cities:
    areas[city]=areamanager.delimiter_area(city)

In [ ]:
cities_pid_in_area=dict()
start_time=time.time()
for city in cities:
    area=areas[city]
    pid_in_area=collections.defaultdict(bool)
    for poi_id in poi_data:
        if areamanager.poi_in_area(area,poi_data[poi_id]):
            pid_in_area[poi_id]=True
    cities_pid_in_area[city]=pid_in_area
print(time.time()-start_time)

In [ ]:
fuser=open("../data/user.json")
user_data = dict()
start_time=time.time()
for i, line in enumerate(fuser):  
    # json to dict
    obj_json = json.loads(line)
    # add to the data collection
    user_data[obj_json['user_id']]=None#obj_json['friends'].split(', ')
user_data_list=list(user_data)
print(time.time()-start_time)

In [ ]:
freview=open("../data/review.json")

cities_checkin_data=dict()
for city in cities:
    cities_checkin_data[city]=list()

start_time=time.time()

for i, line in enumerate(freview):  
    # json to dict
    obj_json = json.loads(line)
    for city in cities:
        if cities_pid_in_area[city][obj_json['business_id']]:
            # add to the data collection
            cities_checkin_data[city].append({'user_id':obj_json['user_id'],
                             'poi_id':obj_json['business_id'],
                             'date':obj_json['date']})
            break
    if i % 500000 ==0:
        print(i)
print(time.time()-start_time)

In [ ]:
ftip=open("../data/tip.json")
start_time=time.time()
for i, line in enumerate(ftip):  
    # json to dict
    for city in cities:
        if cities_pid_in_area[city][obj_json['business_id']]:
            # add to the data collection
            cities_checkin_data[city].append({'user_id':obj_json['user_id'],
                         'poi_id':obj_json['business_id'],
                         'date':obj_json['date']})
            break
    if i % 500000 ==0:
        print(i)
print(time.time()-start_time)

In [ ]:
checkin_data=cities_checkin_data['madison']

In [ ]:
users_id = set()
for check in checkin_data:
    users_id.add(check['user_id'])

In [ ]:
len(users_id)

In [ ]:
pois_id = set()
for check in checkin_data:
    pois_id.add(check['poi_id'])

In [ ]:
len(pois_id)

In [ ]:
training_matrix = np.zeros((len(users_id),len(pois_id)))

In [ ]:
for check in checkin_data:
    training_matrix[user_data_list.index(check['user_id']),poi_data_list.index(check['poi_id']])+=1

In [ ]:
training_matrix